# 利用GPU进行模型训练

In [1]:
#导入第三方库
import torch, torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import time


In [2]:
#准备数据集
train_data = torchvision.datasets.CIFAR10(root='dataset_CIFAR10', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root='dataset_CIFAR10', train=False, transform=torchvision.transforms.ToTensor(), download=True)
test_data_size = len(test_data)
print("测试集的长度为：{}".format(test_data_size))

Files already downloaded and verified
Files already downloaded and verified
测试集的长度为：10000


In [3]:
#使用DataLoader方法加载数据集
train_dataloader = DataLoader(train_data, batch_size= 64, shuffle= True)
test_dataloader = DataLoader(test_data, batch_size= 64, shuffle= True)

In [4]:
#定义训练的设备
device = torch.device("cuda:0")

In [5]:
#第一种利用cuda核心训练神经网络方式：网络模型->数据(输入、标注)->损失函数->.cuda()
#搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x
model = Model()
#先判断cuda核心是否可用
if torch.cuda.is_available():
    model = model.cuda()
    #另外一种方法
    # model = model.to(device)

In [6]:
#使用交叉熵损失函数创建模型的损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.cuda()
#定义优化器
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#优化器没有cuda加速训练的方法

In [8]:
# 添加tensorboard
writer = SummaryWriter(log_dir= "logs_train_model")
start_time = time.time()

#设置训练网络的一些参数
#记录训练的次数
total_train_step = 0
#记录测试的次数
total_test_step = 0
#设置训练的轮数
epochs = 30

for epoch in range(epochs):
    print("-------第{}轮训练开始-------".format(epoch+1))
    #训练步骤
    #在此处可以使用model.train()方法设置模型为训练状态，对模型中的Dropout以及BatchNorm有效
    model.train()
    for data in train_dataloader:
        imgs, targets = data
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            targets = targets.cuda()
            #另一种方法
            # imgs = imgs.to(device)
            # targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs, targets)
        #在进行反向传播优化模型参前，将参数梯度清零
        optimizer.zero_grad()
        #反向传播
        loss.backward()
        #调用优化器进行参数优化
        optimizer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            #记录训练时间
            end_time = time.time()
            print(end_time-start_time)
            # print("训练次数为：{}, Loss: {}".format(total_train_step, loss
            # print("训练次数为：{}, Loss: {}".format(total_train_step, loss)),推荐使用loss.item()方法，将数据类型转变为浮点数
            print("训练次数：{}, 当前Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    #测试步骤
    #在此处可以使用model.eval()方法设置模型为验证状态，对模型中的Dropout以及BatchNorm有效
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets = targets.cuda()
                #另一种方法
                # imgs = imgs.to(device)
                # targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1)== targets).sum()
    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的Accuracy: {}".format(accuracy/test_data_size))
    writer.add_scalar("test_accuracy",accuracy/test_data_size, total_test_step)
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    total_test_step += 1
    #保存模型
    # torch.save(model, "models/model_{}.pth".format(epoch+1))
    #官方推荐的保存方式
    torch.save(model.state_dict(), "models_train/model_{}.pth".format(epoch+1))
    print("模型已保存")

writer.close()

-------第1轮训练开始-------
0.6479146480560303
训练次数：100, 当前Loss: 1.3994017839431763
1.2263579368591309
训练次数：200, 当前Loss: 0.868644118309021
1.8009145259857178
训练次数：300, 当前Loss: 1.2519817352294922
2.3869411945343018
训练次数：400, 当前Loss: 1.3084348440170288
2.9479918479919434
训练次数：500, 当前Loss: 1.2558099031448364
3.5115609169006348
训练次数：600, 当前Loss: 1.3725814819335938
4.06454873085022
训练次数：700, 当前Loss: 0.9270230531692505
整体测试集上的Loss: 183.6374928355217
整体测试集上的Accuracy: 0.0007999999797903001
模型已保存
-------第2轮训练开始-------
5.393904447555542
训练次数：800, 当前Loss: 1.215383529663086
5.966020107269287
训练次数：900, 当前Loss: 1.0193934440612793
6.527115821838379
训练次数：1000, 当前Loss: 0.8646900057792664
7.083083868026733
训练次数：1100, 当前Loss: 0.9046972393989563
7.6292784214019775
训练次数：1200, 当前Loss: 1.1396583318710327
8.1978919506073
训练次数：1300, 当前Loss: 1.0489588975906372
8.765746831893921
训练次数：1400, 当前Loss: 0.9431935548782349
9.341608762741089
训练次数：1500, 当前Loss: 1.157904028892517
整体测试集上的Loss: 178.07402390241623
整体测试集上的Accuracy:

In [15]:
#打印显卡硬件信息
!nvidia-smi

Sun Oct 13 23:02:36 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.09                 Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| 50%   44C    P0             30W /  160W |    1526MiB /   8188MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#第二种使用GPU训练模型的方法 .to(device)
#判断电脑中是否有GPU，如果有则使用GPU，否则使用CPU进行训练
#Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#指定显卡进行训练
Torch.device("cuda:0")